<a href="https://colab.research.google.com/github/Pulisai1704/Python/blob/main/Unit_1_8puzzle_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import heapq
import numpy as np

# Directions for moving tiles (up, down, left, right)
DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]

class Node:
    def __init__(self, puzzle, blank_pos, g, h, parent=None):
        self.puzzle = puzzle
        self.blank_pos = blank_pos
        self.g = g  # Cost from start
        self.h = h  # Heuristic cost
        self.parent = parent

    def __lt__(self, other):
        return (self.g + self.h) < (other.g + other.h)

    def get_f(self):
        return self.g + self.h

def manhattan_distance(puzzle):
    distance = 0
    size = len(puzzle)
    for r in range(size):
        for c in range(size):
            value = puzzle[r][c]
            if value != 0:
                target_r, target_c = divmod(value - 1, size)
                distance += abs(target_r - r) + abs(target_c - c)
    return distance

def is_solvable(puzzle):
    one_d = [tile for row in puzzle for tile in row if tile != 0]
    inversions = 0
    for i in range(len(one_d) - 1):
        for j in range(i + 1, len(one_d)):
            if one_d[i] > one_d[j]:
                inversions += 1
    return inversions % 2 == 0

def generate_moves(node):
    neighbors = []
    size = len(node.puzzle)
    r, c = node.blank_pos

    for dr, dc in DIRECTIONS:
        nr, nc = r + dr, c + dc
        if 0 <= nr < size and 0 <= nc < size:
            new_puzzle = [row[:] for row in node.puzzle]
            new_puzzle[r][c], new_puzzle[nr][nc] = new_puzzle[nr][nc], new_puzzle[r][c]
            new_blank_pos = (nr, nc)
            neighbors.append(Node(new_puzzle, new_blank_pos, node.g + 1, manhattan_distance(new_puzzle), node))

    return neighbors

def print_puzzle(puzzle):
    for row in puzzle:
        print(" ".join(str(tile) for tile in row))
    print()

def solve_puzzle(start_puzzle):
    size = len(start_puzzle)

    if not is_solvable(start_puzzle):
        print("The puzzle is not solvable.")
        return

    start_blank_pos = next((r, c) for r, row in enumerate(start_puzzle) for c, tile in enumerate(row) if tile == 0)
    start_node = Node(start_puzzle, start_blank_pos, 0, manhattan_distance(start_puzzle))

    open_list = []
    heapq.heappush(open_list, start_node)

    closed_list = set()
    closed_list.add(str(start_puzzle))

    while open_list:
        current_node = heapq.heappop(open_list)

        if current_node.h == 0:
            print_solution(current_node)
            return

        for neighbor in generate_moves(current_node):
            state_str = str(neighbor.puzzle)
            if state_str not in closed_list:
                closed_list.add(state_str)
                heapq.heappush(open_list, neighbor)

    print("No solution found.")

def print_solution(node):
    if node is None:
        return
    print_solution(node.parent)
    print_puzzle(node.puzzle)

if __name__ == "__main__":
    puzzle = [
        [1, 2, 3],
        [4, 5, 6],
        [7, 0, 8]
    ]

    solve_puzzle(puzzle)


1 2 3
4 5 6
7 0 8

1 2 3
4 5 6
7 8 0

